In [51]:
import pandas as pd
from src.utils import EncoderDecoder, plot_climb
import torch
from src.data.datasets import KilterDataset
import plotly.express as px
import numpy as np
from typing import Union

In [37]:
df = pd.read_csv("data/raw/all_climbs.csv", index_col=0)
holds = pd.read_csv("data/raw/holds.csv", index_col=0)

In [80]:
class EncoderDecoder:
    """Converts frames to tensors and back"""

    def __init__(self, holds: pd.DataFrame):
        self.holds = holds
        self.coord_to_id = self._create_coord_to_id()
        self.id_to_coord = self._create_id_to_coord()

    def _create_coord_to_id(self):
        hold_lookup_matrix = np.zeros((48, 48), dtype=int)
        for i in range(48):
            for j in range(48):
                hold = self.holds[
                    (self.holds["x"] == (i * 4 + 4)) & (self.holds["y"] == (j * 4 + 4))
                ]
                if not hold.empty:
                    hold_lookup_matrix[i, j] = int(hold.index[0])
        return hold_lookup_matrix

    def _create_id_to_coord(self):
        id_to_coord = self.holds[["x", "y"]]
        id_to_coord = (id_to_coord - 4) // 4
        return id_to_coord.transpose().to_dict(orient="list")

    def str_to_tensor(self, frames: str, angle: float) -> torch.Tensor:
        angle_matrix = torch.ones((1, 48, 48), dtype=torch.float32) * (angle / 70)
        matrix = torch.zeros((4, 48, 48), dtype=torch.float32)
        for frame in frames.split("p")[1:]:
            hold_id, color = frame.split("r")
            hold_id, color = int(hold_id), int(color) - 12
            coords = self.id_to_coord[hold_id]
            matrix[color, coords[0], coords[1]] = 1
        return torch.cat((matrix, angle_matrix), dim=0)

    def tensor_to_str(self, matrix: torch.Tensor) -> str:
        angle = (matrix[-1].mean() * 70).round().long().item()
        matrix = matrix[:-1, :, :]
        frames = []
        counter = [0,0,0,0]
        for color, x, y in zip(*torch.where(matrix)):
            counter[color] += 1
            color, x, y = color.item(), x.item(), y.item()
            if counter[color] > 2 and color in [0,2]:
                continue
            hold_id = self.coord_to_id[x, y]
            role = color + 12
            frames.append((hold_id, role))
        sorted_frames = sorted(frames, key=lambda x: x[0])
        return (
            "".join([f"p{hold_id}r{role}" for hold_id, role in sorted_frames]), angle
        )

    def __call__(self, *args):
        if len(args) == 1:
            return self.tensor_to_str(*args)
        elif len(args) == 2:
            return self.str_to_tensor(*args)
        else:
            raise ValueError(f"Only 2 input args allowed! You provided {len(args)}")


encdec = EncoderDecoder(holds)


In [82]:
frames = df.iloc[0]["frames"]
print(frames)
t = encdec(frames, 40)
t[0,40,40] = 1
print(encdec(t)[0])


p1100r15p1103r15p1137r15p1151r15p1168r12p1178r15p1187r12p1192r15p1197r15p1200r15p1220r13p1246r13p1253r13p1265r13p1268r13p1278r14
p1100r15p1103r15p1137r15p1151r15p1168r12p1178r15p1187r12p1192r15p1197r15p1200r15p1220r13p1246r13p1253r13p1265r13p1268r13p1278r14
